## Question 2

Develop a Python program for a Fraud Detection System that analyses a file containing credit card information, timestamps, and locations to identify potential fraud alerts? You are provided with a dataset of credit card transactions. How can the program identify suspicious activities, such as multiple transactions from different locations which might be 20 or more Kilometers more apart and within 30 mins ? You can use any third part API to calculate the distance between two locations. [20]

**Example - transactions.csv**

| Card Number               | Amount      | Location       | Date/Time         |
| -------------------------| ----------- | -------------- | ----------------- |
| 4367-7878-8898-8989      | INR 1000    | Gurgaon        | 27/03/23 17:58    |
| 4367-7878-8898-8189      | INR 2000    | Noida          | 27/03/23 17:59    |
| 4367-7878-8898-8989      | INR 5000    | Noida          | 27/03/23 18:10    |

Output: List of fraud entries
```
[
  ['4367-7878-8898-8989', 'INR 1000', 'Gurgaon', '27/03/23 17:58'],
  ['4367-7878-8898-8989', 'INR 5000', 'Noida', '27/03/23 18:10']
]
```


### Additional information
1. For installation of libraries refer - https://colab.research.google.com/notebooks/snippets/importing_libraries.ipynb
2. Sample input file will be located at path `input/q2/transactions.csv`
3. Csv file is a text file where each row has comma separated values.
4. APIs that can be used to get coordinates of location
  * OpenCage APIs - https://opencagedata.com/api
  * How to use this api - follow quick start steps here https://opencagedata.com/api
     1. Signup
     2. Generate your API key - https://opencagedata.com/dashboard#geocoding
     3. Use api https://api.opencagedata.com/geocode/v1/json?q={place}&key={OPENCAGEDATA_KEY}' in your code to get the coordinates for a location, replace `{place}` with name of the place and `{OPENCAGEDATA_KEY}` with your generated api-key in the api request.


5. You can use library `geodesic` to calculate distance between two location coordinates, install by using command block `!pip install geodesic`



In [1]:
import csv

with open('input/q2/transactions.csv', mode='r') as file:
    reader = csv.reader(file)
    for line in reader:
        # Process each line in the csv here
        print(line)

['4367-7878-8898-8989', 'INR 1000', 'Gurgaon', '27/03/23 17:58']
['4367-7878-8898-8189', 'INR 2000', 'Noida', '27/03/23 17:59']
['4367-7878-8898-8989', 'INR 5000', 'Noida', '27/03/23 18:10']
